### Supervised learning algorithms:

supervised learning model:

- Use FY2015 icd9 diagnosis code long desciption as training data.
- Use "Other Diagnosis" column from Excel as test data

- Import Traning data, Separate input and output by slicing into 2 the arrays: one is input for training, the other is target. 
- Prepare y output target. Use sklearn.preprocessing.LabelEncoder to encode the Target which is ctr of icd9 dx code.
- Prepare X input array. Perform feature extraction by using klearn.feature_extraction.text  CountVectorizer with common "stop words" (ENGLISH_STOP_WORDS) against icd9 dx long desc.
- Create a corpus (bag-of-words with their usage statistics) from the terms that occur answers.
- Enhanced the input variable X with a weight for every term. Use klearn.feature_extraction.text import TfidfVectorizer
- Use class sklearn.svm.SVC as estimator to fit X/y to implements vector classification.

- Prepare X-test data ("Other Diagnosis") the same way as the training datasets, using TfidfVectorizer.
- Predict X_test with the model created above.





As tf–idf is very often used for text features, there is also another class called TfidfVectorizer that combines all the options of CountVectorizer and TfidfTransformer in a single model:

In [ ]:
#>>> from sklearn.feature_extraction.text import TfidfVectorizer
#>>> vectorizer_t = TfidfVectorizer(stop_words=stop_words.ENGLISH_STOP_WORDS)
#>>> X = vectorizer_t.fit_transform(corpus)
#>>> X.toarray()

In [ ]:
#>>> vectorizer_t.transform(['Something completely dystrophy.']).toarray()

### Prepare Sample X_test

In [5]:
import pandas as pd
import numpy as np
#filename = '~/Downloads/icd9dxref.fy15.txt'
url="https://raw.githubusercontent.com/azknowesis/team-collaboration/master/NLP/oth_dx.txt"
df_test = pd.read_csv(url, header=None, delimiter='|', dtype=str, nrows = 5000)
df_test.shape
# print(df)

(5000, 2)

In [1]:
>>> from joblib import dump, load

# load vector transformer
>>> vectorizer_t = load('./joblib_dump/vectroizer_t.pkl')

# load label encoder
>>> le = load('./joblib_dump/le_trnfit.joblib')

# load classifier
>>> clf = load('./joblib_dump/sklearn_svm/clf_svclinear_cache500_trnfit.joblib')
# >>> clf = load('./joblib_dump/sklearn_svm/clf_svm_svc_15000_trnfit.joblib')
# >>> clf = load('./joblib_dump/sklearn_MultinomialNB/clf_MultinomialNB_15000_trnfit.joblib')


In [2]:
# inputx_test = ['blind hypotensive eye',
# 'eye sensitivity to light photophobia',
# 'concussion with no loss of consciousness',
# 'photophobia od',
# 'h/o tbi with chronic photophobia esp od.sxs have improved over past year. ct orbits was normal still symptomatic sunglass help.',
# 'pdx traumatic vision loss od',
# 'suspect corneal abrasion due to irritation from contact lense',
# 'pdx photophobia right']

# #inputx_test

# # transform and vecterize input dx string based on training model
# X_test = vectorizer_t.transform(inputx_test)
# X_test.toarray()
# X_test.shape
    
# # predict, and get dx code based on encoder value
# # clf.predict(X_test)
# outputx_test = list(le.inverse_transform(clf.predict(X_test)))
# outputx_test

['36041', '36813 1', '8500', '36813 1', '36813 1', 'V720', '52120', '36813 1']

In [6]:
# export output
pathname = './flatfiles/svmsvclinear/'
# pathname = './flatfiles/sklearn_svmsvcrbf/'
# pathname = './flatfiles/sklearn_MultinomialNB/'

batch_size = 5000
n=0

#for n in range(1): # range 2 includes 0:1
while n < 1:
    df_a = df_test[batch_size*n:batch_size*(n+1)]  # used to  result = pd.concat([df_a, s1], axis=1) in the end
    df_test1 = df_test.iloc[batch_size*n:batch_size*(n+1),1]
    print(df_test1)
    #df_test1.shape
    
    # Convert Dataframe to list
    inputx_test = df_test1.astype(str).values.tolist()
    #inputx_test
    
    # transform and vecterize input dx string based on training model
    X_test = vectorizer_t.transform(inputx_test)
    X_test.toarray()
    X_test.shape
    
    # predict, and get dx code based on encoder value
    # clf.predict(X_test)
    outputx_test = list(le.inverse_transform(clf.predict(X_test)))
    #outputx_test
    
    # Option 3:
    #df_a[2] = outputx_test, got SettingWithCopyWarning. use df.assign instead
    result = df_a.assign(e=outputx_test)
    
    # export output

    filename = pathname + 'batch_' + str(n)
    result.to_csv(filename, sep='|', header=False)
    
    n += 1

0                                   blind hypotensive eye
1                                                headache
2                concussion with no loss of consciousness
3                                     epiretinal membrane
4       subretinal and intraretinal hemorrhage od with...
5                            pdx traumatic vision loss od
6       suspect corneal abrasion due to irritation fro...
7                                          pseudo aphakia
8       s/p os 25c ppv/ilm el with icg/ivf/5f6 20% 1/8...
9                                                pciol od
10                   eye sensitivity to light photophobia
11      s/p os 256ppv/icm peel+fc6/ivf/sf6 20% macular...
12                                          eye trauma od
13                                      macular puckering
14                                       dry eye syndrome
15                   visit for military services physical
16                         s/p os 256 ppv/ic6/ivt/sf6 25%
17            

In [ ]:
# Option 1:
#>>> import pandas as pd
#>>> dict1 = {'oth_dx':inputx_test,'dx_code':outputx_test}
#>>> df_output = pd.DataFrame(dict1)
#>>> df_output

# Option 2: got warning
# df_b = pd.DataFrame(outputx_test)
#result = pd.concat([df_a, df_b], axis=1)

In [ ]:
#>>> result = df1.append([df2, df3])